# Step 1: Imports and Setup

In [1]:
# %%
import duckdb
import pandas as pd
from pathlib import Path

# --- Configuration ---
# Point to the database file you created in the ETL step
OUTPUT_DIR = Path("../output")
DB_FILE = OUTPUT_DIR / "synthea_fhir.duckdb"

# --- Connect to the database ---
# We're connecting in read-only mode since we are just querying, not writing new tables.
print(f"Connecting to DuckDB database: {DB_FILE}")
con = duckdb.connect(database=str(DB_FILE), read_only=True)

# Set pandas to display more columns for our sanity checks
pd.set_option('display.max_columns', 50)

Connecting to DuckDB database: ../output/synthea_fhir.duckdb


# Step 2: Identify Index Admissions

Our goal is to predict readmission following an inpatient hospital stay. We can't predict readmissions for every single encounter (like a regular doctor's visit). We must first create a base table of only the relevant hospitalizations. These are our index admissions.

In the Synthea dataset, inpatient encounters are marked with the code 'IMP' in the EncounterClass column.

In [2]:
# %%
# SQL query to select all inpatient encounters
sql_query = """
SELECT
    Id AS encounter_id,
    Patient AS patient_id,
    Start AS admission_date,
    Stop AS discharge_date
FROM
    encounters
WHERE
    EncounterClass = 'IMP'
"""

# Execute and fetch into a pandas DataFrame
index_admissions_df = con.execute(sql_query).fetchdf()

🩺 Sanity Check
Let's verify that we've correctly filtered the data. We should see a reasonable number of inpatient stays and the dates should look correct.

In [3]:
# %%
print(f"Found {len(index_admissions_df):,} total inpatient admissions.")
print("First 5 index admissions:")
display(index_admissions_df.head())

# Check for any admissions with null discharge dates, as these are problematic
print(f"\nAdmissions with missing discharge dates: {index_admissions_df['discharge_date'].isnull().sum()}")

Found 104,068 total inpatient admissions.
First 5 index admissions:


,encounter_id,patient_id,admission_date,discharge_date
0,d9f75434-be7a-663f-8702-8ac72afe10fd,7b0e0003-89c8-6020-da41-033cf174c76f,1997-05-18 10:35:30,1997-05-23 08:41:12
1,58fa79a0-fb0a-9eb3-562a-ece296e084f6,e8e952cb-c390-f661-ad04-b95818ec40a4,1994-05-21 00:52:34,1994-05-25 19:53:04
2,8be2ca69-121a-a779-3ac9-2ecb97ad8908,49b95517-d094-9b08-3982-c86d39450368,2025-04-03 20:25:55,2025-04-04 20:40:55
3,ee89a84f-9b21-fd36-2419-9d85ce9f8b97,4c161043-b934-badf-3375-019f05f72cb1,2016-03-08 00:08:09,2016-03-22 01:08:09
4,54ce90f5-1bcd-0379-2c0c-d000a57e390c,4c161043-b934-badf-3375-019f05f72cb1,2016-11-08 00:08:09,2016-11-13 00:08:09



Admissions with missing discharge dates: 0


# Step 3: Engineer the Target Variable (readmitted_within_30_days)

This is the most critical step. For each index admission, we need to determine if another inpatient admission for the same patient occurred within 30 days of their discharge.

We can do this efficiently using a SQL window function (LEAD) to find the next admission date for each patient.

In [4]:
# %%
# This query calculates the time to the next admission and creates the readmission flag.
sql_query = """
WITH PatientAdmissions AS (
    -- 1. Get all inpatient admissions and order them by patient and date
    SELECT
        Id AS encounter_id,
        Patient AS patient_id,
        Start AS admission_date,
        Stop AS discharge_date,
        -- 2. Use the LEAD window function to get the start date of the *next* admission
        -- for the same patient. If there is no next admission, it will be NULL.
        LEAD(Start, 1) OVER(PARTITION BY Patient ORDER BY Start) AS next_admission_date
    FROM
        encounters
    WHERE
        EncounterClass = 'IMP'
)
-- 3. Now, calculate the difference and create the flag
SELECT
    encounter_id,
    patient_id,
    admission_date,
    discharge_date,
    next_admission_date,
    -- Calculate days between discharge and the next admission
    DATE_DIFF('day', discharge_date, next_admission_date) AS days_to_next_admission,
    -- Create the binary target variable
    CASE
        WHEN DATE_DIFF('day', discharge_date, next_admission_date) <= 30 THEN 1
        ELSE 0
    END AS readmitted_within_30_days
FROM
    PatientAdmissions
-- We can't predict readmission for ongoing hospitalizations
WHERE
    discharge_date IS NOT NULL
"""

# This will be our main dataset that we'll add features to
readmissions_df = con.execute(sql_query).fetchdf()

🩺 Sanity Check
Let's check the distribution of our new target variable. We should also manually inspect a few positive cases to be sure our logic is correct.

In [5]:
# %%
# Check the distribution of the target variable
print("Readmission distribution:")
print(readmissions_df['readmitted_within_30_days'].value_counts(normalize=True))

# Find a patient who was readmitted and verify the dates manually
readmitted_patient_example = readmissions_df[readmissions_df['readmitted_within_30_days'] == 1].head(5)
print("\nExample of patients who were readmitted:")
display(readmitted_patient_example)

Readmission distribution:
readmitted_within_30_days
0    0.821905
1    0.178095
Name: proportion, dtype: float64

Example of patients who were readmitted:


,encounter_id,patient_id,admission_date,discharge_date,next_admission_date,days_to_next_admission,readmitted_within_30_days
4,468f6e2a-27ed-bf00-c788-cac815bbcb6d,f8250361-7019-9ab3-0dde-ee294cee0932,2009-12-10 12:27:53,2009-12-11 17:03:53,2009-12-11 17:03:53,0,1
21,7b05de6c-0a13-1777-470b-6c01eab410ac,f92afa89-7958-d7de-523d-d6bd02ac1c31,2019-09-30 00:46:24,2019-10-02 02:47:27,2019-10-02 02:32:27,0,1
58,815688e0-390e-90e2-7f50-75881fbc46c2,fa83a4b5-54a2-24cc-1e30-82894488754c,2007-02-23 10:07:58,2007-02-25 12:51:58,2007-02-25 12:51:58,0,1
66,279abd20-f01e-025c-ea10-024690269314,faced69b-873c-c386-9a8f-b6851fcc47b1,2021-08-16 22:08:21,2021-08-23 04:30:56,2021-08-23 22:08:21,0,1
87,77fd2431-3d67-be14-45a2-5c8de33c9e31,fb2388d2-ecd6-972d-59ce-38d7683fd37c,2017-08-16 18:31:40,2017-08-23 09:21:40,2017-08-23 09:21:40,0,1


# Step 4: Add Demographics and Admission-Level Features

Now we can start building our features. We'll join our readmissions_df with the patients table to get demographic data and also calculate some features from the admission itself.

In [6]:
# %%
sql_query = """
SELECT
    -- Key identifiers from our readmissions table
    readmissions.encounter_id,
    readmissions.patient_id,

    -- The target variable
    readmissions.readmitted_within_30_days,

    -- Feature 1: Length of Stay (in days)
    DATE_DIFF('day', readmissions.admission_date, readmissions.discharge_date) AS length_of_stay,

    -- Feature 2: Age at time of admission
    DATE_DIFF('year', patients.BirthDate, readmissions.admission_date) AS age_at_admission,

    -- Demographic features from the patients table
    patients.Gender AS gender,
    patients.Race AS race,
    patients.Marital AS marital_status,
    
    -- Admission details from the original encounters table
    enc.Description AS admission_reason,
    enc.ReasonDescription AS admission_reason_detail

FROM
    -- Use our previously created DataFrame as a source table
    readmissions_df AS readmissions
LEFT JOIN
    patients ON readmissions.patient_id = patients.Id
LEFT JOIN
    encounters AS enc ON readmissions.encounter_id = enc.Id
"""

# Create our analytical base table
model_df = con.execute(sql_query).fetchdf()

🩺 Sanity Check
Let's inspect the resulting dataframe. The row count should be the same as our readmissions_df. We should also check the distributions of our new numeric features like age_at_admission and length_of_stay to see if they are reasonable.

In [7]:
# %%
print(f"Total rows in our model dataset: {len(model_df):,}")
print("First 5 rows of the feature table:")
display(model_df.head())

print("\nStatistical summary of numeric features:")
display(model_df[['length_of_stay', 'age_at_admission']].describe())

Total rows in our model dataset: 104,068
First 5 rows of the feature table:


,encounter_id,patient_id,readmitted_within_30_days,length_of_stay,age_at_admission,gender,race,marital_status,admission_reason,admission_reason_detail
0,138fa8d2-c18d-18f6-2d10-94511e19edbc,79f75722-a60e-c575-a7af-4da0ae591a52,0,11,2,male,None,S,Hospital admission (procedure),Patient transfer to skilled nursing facility (...
1,edbab4b3-26b2-fcd7-4401-e3622cbfee4d,f65c8b52-a710-5ec4-0c26-44b15d1174a5,0,2,38,male,None,W,Drug rehabilitation and detoxification (regime...,Dependent drug abuse (disorder)
2,46939e78-cb1b-df7f-9202-37bef958c60c,f65c8b52-a710-5ec4-0c26-44b15d1174a5,0,4,40,male,None,W,Drug rehabilitation and detoxification (regime...,Dependent drug abuse (disorder)
3,dc23e269-a1c9-a52a-9558-28cd57391e6f,f65c8b52-a710-5ec4-0c26-44b15d1174a5,0,3,41,male,None,W,Drug rehabilitation and detoxification (regime...,Dependent drug abuse (disorder)
4,1ab103bf-9d29-cb4f-19c6-d7f2c11391bf,f65c8b52-a710-5ec4-0c26-44b15d1174a5,0,4,47,male,None,W,Drug rehabilitation and detoxification (regime...,Dependent drug abuse (disorder)



Statistical summary of numeric features:


,length_of_stay,age_at_admission
count,104068.000000,104068.000000
mean,6.917170,50.162759
std,14.102942,19.497225
min,0.000000,0.000000
25%,1.000000,36.000000
50%,4.000000,52.000000
75%,8.000000,65.000000
max,3306.000000,111.000000


# Step 5: Engineer Historical Features (Advanced)

In [8]:
# %%
# This query is more complex. It performs a "self-join" on the encounters
# table to count previous visits for each index admission.
sql_query = """
SELECT
    index_admission.Id AS encounter_id,
    -- Count all previous inpatient admissions within the last 365 days
    COUNT(prior_admissions.Id) AS prior_admissions_last_year
FROM
    encounters AS index_admission
LEFT JOIN
    encounters AS prior_admissions
ON
    -- Must be the same patient
    index_admission.Patient = prior_admissions.Patient
    -- The prior admission must have happened BEFORE the index one
    AND prior_admissions.Start < index_admission.Start
    -- And it must be within the last year
    AND DATE_DIFF('day', prior_admissions.Start, index_admission.Start) <= 365
    -- And it must also be an inpatient admission
    AND prior_admissions.EncounterClass = 'IMP'
WHERE
    -- We only need to calculate this for our index admissions
    index_admission.EncounterClass = 'IMP'
GROUP BY
    index_admission.Id;
"""

prior_admissions_df = con.execute(sql_query).fetchdf()

# Now, merge this back into our main model dataframe
model_df = pd.merge(model_df, prior_admissions_df, on='encounter_id', how='left')

# Fill any NaNs that might result from the merge with 0
model_df['prior_admissions_last_year'] = model_df['prior_admissions_last_year'].fillna(0)

🩺 Sanity Check
Let's check the new column. The values should be integers, and the distribution should make sense (most patients will likely have 0 or 1 prior admissions).

In [9]:
# %%
print("Distribution of prior admissions in the last year:")
print(model_df['prior_admissions_last_year'].value_counts().sort_index())

display(model_df.head())

Distribution of prior admissions in the last year:
prior_admissions_last_year
0     66546
1     10832
2      3184
3      1464
4       860
5       804
6       928
7      1214
8      3005
9     10674
10     1931
11     1944
12      602
13       64
14       16
Name: count, dtype: int64


,encounter_id,patient_id,readmitted_within_30_days,length_of_stay,age_at_admission,gender,race,marital_status,admission_reason,admission_reason_detail,prior_admissions_last_year
0,138fa8d2-c18d-18f6-2d10-94511e19edbc,79f75722-a60e-c575-a7af-4da0ae591a52,0,11,2,male,None,S,Hospital admission (procedure),Patient transfer to skilled nursing facility (...,0
1,edbab4b3-26b2-fcd7-4401-e3622cbfee4d,f65c8b52-a710-5ec4-0c26-44b15d1174a5,0,2,38,male,None,W,Drug rehabilitation and detoxification (regime...,Dependent drug abuse (disorder),0
2,46939e78-cb1b-df7f-9202-37bef958c60c,f65c8b52-a710-5ec4-0c26-44b15d1174a5,0,4,40,male,None,W,Drug rehabilitation and detoxification (regime...,Dependent drug abuse (disorder),0
3,dc23e269-a1c9-a52a-9558-28cd57391e6f,f65c8b52-a710-5ec4-0c26-44b15d1174a5,0,3,41,male,None,W,Drug rehabilitation and detoxification (regime...,Dependent drug abuse (disorder),0
4,1ab103bf-9d29-cb4f-19c6-d7f2c11391bf,f65c8b52-a710-5ec4-0c26-44b15d1174a5,0,4,47,male,None,W,Drug rehabilitation and detoxification (regime...,Dependent drug abuse (disorder),0


# Step 5b: Engineer Clinical Features (Diagnoses, Procedures, Medications)

In [10]:
# %%
# ----- 1. Number of Diagnoses (Conditions) -----
print("Engineering feature: Number of Diagnoses...")
sql_diagnoses = """
SELECT
    Encounter AS encounter_id,
    COUNT(Code) AS num_diagnoses
FROM
    conditions
GROUP BY
    Encounter;
"""
diagnoses_df = con.execute(sql_diagnoses).fetchdf()

# Merge into the main dataframe
model_df = pd.merge(model_df, diagnoses_df, on='encounter_id', how='left')
model_df['num_diagnoses'] = model_df['num_diagnoses'].fillna(0)


# ----- 2. Number of Procedures -----
print("Engineering feature: Number of Procedures...")
sql_procedures = """
SELECT
    Encounter AS encounter_id,
    COUNT(Code) AS num_procedures
FROM
    procedures
GROUP BY
    Encounter;
"""
procedures_df = con.execute(sql_procedures).fetchdf()

# Merge into the main dataframe
model_df = pd.merge(model_df, procedures_df, on='encounter_id', how='left')
model_df['num_procedures'] = model_df['num_procedures'].fillna(0)


# ----- 3. Number of Medications -----
print("Engineering feature: Number of Medications...")
sql_medications = """
SELECT
    Encounter AS encounter_id,
    COUNT(Code) AS num_medications
FROM
    medications
GROUP BY
    Encounter;
"""
medications_df = con.execute(sql_medications).fetchdf()

# Merge into the main dataframe
model_df = pd.merge(model_df, medications_df, on='encounter_id', how='left')
model_df['num_medications'] = model_df['num_medications'].fillna(0)

print("✅ Clinical features added successfully.")

Engineering feature: Number of Diagnoses...
Engineering feature: Number of Procedures...
Engineering feature: Number of Medications...
✅ Clinical features added successfully.


🩺 Sanity Check
Let's inspect our dataframe again to ensure the new columns have been added correctly. They should contain integer counts.

In [11]:
# %%
print("First 5 rows with new clinical features:")
display(model_df.head())

print("\nUpdated statistical summary:")
display(model_df[['length_of_stay', 'age_at_admission', 'prior_admissions_last_year', 'num_diagnoses', 'num_procedures', 'num_medications']].describe())

First 5 rows with new clinical features:


,encounter_id,patient_id,readmitted_within_30_days,length_of_stay,age_at_admission,gender,race,marital_status,admission_reason,admission_reason_detail,prior_admissions_last_year,num_diagnoses,num_procedures,num_medications
0,138fa8d2-c18d-18f6-2d10-94511e19edbc,79f75722-a60e-c575-a7af-4da0ae591a52,0,11,2,male,None,S,Hospital admission (procedure),Patient transfer to skilled nursing facility (...,0,0.0,20.0,0.0
1,edbab4b3-26b2-fcd7-4401-e3622cbfee4d,f65c8b52-a710-5ec4-0c26-44b15d1174a5,0,2,38,male,None,W,Drug rehabilitation and detoxification (regime...,Dependent drug abuse (disorder),0,0.0,0.0,0.0
2,46939e78-cb1b-df7f-9202-37bef958c60c,f65c8b52-a710-5ec4-0c26-44b15d1174a5,0,4,40,male,None,W,Drug rehabilitation and detoxification (regime...,Dependent drug abuse (disorder),0,0.0,0.0,0.0
3,dc23e269-a1c9-a52a-9558-28cd57391e6f,f65c8b52-a710-5ec4-0c26-44b15d1174a5,0,3,41,male,None,W,Drug rehabilitation and detoxification (regime...,Dependent drug abuse (disorder),0,0.0,0.0,0.0
4,1ab103bf-9d29-cb4f-19c6-d7f2c11391bf,f65c8b52-a710-5ec4-0c26-44b15d1174a5,0,4,47,male,None,W,Drug rehabilitation and detoxification (regime...,Dependent drug abuse (disorder),0,0.0,0.0,0.0



Updated statistical summary:


,length_of_stay,age_at_admission,prior_admissions_last_year,num_diagnoses,num_procedures,num_medications
count,104068.000000,104068.000000,104068.000000,104068.000000,104068.000000,104068.000000
mean,6.917170,50.162759,2.039032,0.377984,9.333936,0.183995
std,14.102942,19.497225,3.557021,0.719728,15.938124,0.818711
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,36.000000,0.000000,0.000000,0.000000,0.000000
50%,4.000000,52.000000,0.000000,0.000000,4.000000,0.000000
75%,8.000000,65.000000,2.000000,1.000000,12.000000,0.000000
max,3306.000000,111.000000,14.000000,8.000000,176.000000,9.000000


# Step 6: Save the Final Dataset & Cleanup

Our feature engineering is done for now! We have a clean, single table ready for the next steps. It's best practice to save this intermediate result to a fast, efficient file format like Parquet.

In [12]:
# %%
# Define the path for our final dataset
MODEL_DATA_FILE = OUTPUT_DIR / "readmissions_dataset.parquet"

# Save to Parquet
model_df.to_parquet(MODEL_DATA_FILE, index=False)

# Close the database connection
con.close()

print(f"✅ Successfully created and saved the feature table with {len(model_df):,} rows and {len(model_df.columns)} columns.")
print(f"Dataset saved to: {MODEL_DATA_FILE}")

✅ Successfully created and saved the feature table with 104,068 rows and 14 columns.
Dataset saved to: ../output/readmissions_dataset.parquet
